# Sagehen Creek pynhm + mf6
This notebook reproduces the results of 

Hughes, Joseph D., Martijn J. Russcher, Christian D. Langevin, Eric D. Morway, and Richard R. McDonald. "The MODFLOW Application Programming Interface for simulation control and software interoperability." Environmental Modelling & Software 148 (2022): 105257.

for the Coupling MODFLOW to PRMS section. Here we substitute pynhm for PRMS-BMI used in the paper. This coupling was demonstrated in our [AGU 2022 poster](https://agu2022fallmeeting-agu.ipostersessions.com/default.aspx?s=05-E1-C6-40-DF-0D-4D-C7-4E-DE-D2-61-02-05-8F-0A).

This notebook is meant to be run inside the repository for that paper because that's where the data are 

https://github.com/jdhughes-usgs/mf6bmipaper

as will be setup below. You'll need to clone the above repo and specify its location below. (We wont add this notebook to that repository, since it's not part of the paper.) 

You may need to obtain a MODFLOW 6 DLL for your platform from https://github.com/MODFLOW-USGS/modflow6/releases, the DLL in the mf6bmipaper repo may not work for you. Changes (hopefully bug fixes) in the DLL may change the overall results somewhat.

The python environment dependencies for this notebook should be completely specified in environment.yaml found in this directoy. You can see how to create a conda environment form this file in the example notebook `../00_python_virtual_env.ipynb`: `conda env create -f examples_env.yml`.

For plotting output, run the plotting notebooks in this directory: `sagehen-postprocess-graphs.ipynb` and `sagehen-postprocess-maps.ipynb`.

In [ ]:
import os
import pathlib as pl
import shutil
import sys

import numpy as np
import netCDF4 as nc
from modflowapi import ModflowApi
import pynhm

In [ ]:
# Set this to your path for this repo
root_dir = pl.Path('/Users/jamesmcc/usgs/mf6bmipaper/models/ModflowPynhm/')
# Set this to the path to the repos where this current notebook is located
pynhm_dir = pl.Path('/Users/jamesmcc/usgs/pynhm_2/')

In [ ]:
# Also configure this to where your DLL is found
if sys.platform == "win32":
    mf6_dll = root_dir.parent / '/bin/libmf6.dll'
else:
    mf6_dll = root_dir.parent / 'bin/libmf6.dylib'
assert mf6_dll.exists()    

In [ ]:
# Create the run directory
name = "sagehenmodel"
run_dir = root_dir / name
if not run_dir.exists(): 
    run_dir.mkdir(parents=True)

os.chdir(run_dir)
assert run_dir.exists()
print(os.getcwd())

### Read weights

The weight matrix should have columns equal to the number of HRUs and rows equal to the number of UZF cells or number of SFR reaches.

_UZF weights_

In [ ]:
shutil.copy2(root_dir.parent / 'ModflowPRMS/weights.npz', root_dir / 'weights.npz')
uz2 = np.load(root_dir / 'weights.npz')
print(uz2['uzfw'].shape, uz2['sfrw'].shape)
uzfw = uz2['uzfw']

_SFR weights_

In [ ]:
sfrw = uz2['sfrw']

_Number of UZF cells at the top of the model_

In [ ]:
nuzf_infilt = uzfw.shape[0]
print("number of UZF cells at the top of the model {}".format(nuzf_infilt))

### Function to map HRU values to MODFLOW 6 values

In [ ]:
def hru2mf6(weights, values):
    return weights.dot(values)

### Run loosely coupled PRMS and MODFLOW 6 models

#### Initialize pynhm components

In [ ]:
# Have to bring in or create some file

# This parameter dictionary was adapted from PRMS6 parameter file
param_file = root_dir / 'sagehen_params.pkl'
_ = shutil.copy2(pynhm_dir / 'examples/sagehen/sagehen_params.pkl', param_file)

# Control file
control_file = root_dir / 'pynhm.control'
_ = shutil.copy2(pynhm_dir / 'examples/sagehen/pynhm.control', control_file)

In [ ]:
# PRMS forcings need converted to netcdf for pynhm

import pickle
with open(param_file, "rb") as input_file:
     param_dict = pickle.load(input_file)

params = pynhm.PrmsParameters(param_dict)

src_dir = root_dir.parent / 'ModflowPRMS/sagehenmodel/surf_climate'
tgt_dir = root_dir / 'pynhm_climate'
tgt_dir.mkdir(exist_ok=True)

cbh_files = {
    src_dir / 'precip.day': tgt_dir / 'prcp.nc',
    src_dir / 'tmax.day': tgt_dir / 'tmax.nc',
    src_dir / 'tmin.day': tgt_dir / 'tmin.nc',
}

from pynhm.utils.cbh_utils import cbh_files_to_netcdf

for src, tgt in cbh_files.items():
    var_name = tgt.with_suffix('').name
    if tgt.exists():
        print(f"output file already exists, skipping: {tgt}")
        continue
    else: 
        print(f"creating {tgt}")
        
    cbh_files_to_netcdf({var_name: src}, params, tgt)        


In [ ]:
# This step is critical for MF6 initialization
output_dir = run_dir / 'output'
if not output_dir.exists():
    output_dir.mkdir()

input_dir = tgt_dir
for ff in [param_file, control_file, input_dir, output_dir]: 
    assert ff.exists()

In [ ]:
params = pynhm.PrmsParameters(parameter_dict=param_dict)
control = pynhm.Control.load(control_file, params=params)

# Only need a PRMS/NHM model through soilzone
prms = pynhm.Model(
    pynhm.PRMSSolarGeometry, 
    pynhm.PRMSAtmosphere, 
    pynhm.PRMSCanopy, 
    pynhm.PRMSSnow,
    pynhm.PRMSRunoff,
    pynhm.PRMSSoilzone,
    control=control, 
    input_dir=input_dir, 
    budget_type='warn')

# prms.initialize_netcdf(output_dir)
# can this just be set up as adapters on pynhm output?

#### Calculate multipliers for PRMS internal variables

In [ ]:
m2ft = 3.28081
in2m = 1. / (12. * m2ft)
acre2m2 = 43560. / (m2ft * m2ft)

In [ ]:
hru_area_m2 = params.parameters["hru_area"] * acre2m2

#### Create arrays to save results

In [ ]:
ntimes = int(control.n_times)
print("Number of days to simulate {}".format(ntimes))

nhm_vars = ['ppt_out', 'actet_out', 'potet_out', 'soilinfil_out', 'runoff_out', 'interflow_out']
nhm_var_dict = {}
nhm_var_dict['time_out'] = np.empty(ntimes,dtype='datetime64[s]')
for vv in nhm_vars: 
    nhm_var_dict[vv] = np.zeros((ntimes, hru_area_m2.shape[0]), dtype=np.float64)

#### Initialize MODFLOW 6

In [ ]:
(root_dir / 'common').mkdir(exist_ok=True)
cp_list = [
    'sagehenmodel/mfsim.nam',
    'sagehenmodel/ex-gwf-sagehen-gsf.tdis',
    'sagehenmodel/gwf_sagehen-gsf.nam',
    'sagehenmodel/gwf_sagehen-gsf.ic',
    'sagehenmodel/gwf_sagehen-gsf.sto',
    'sagehenmodel/gwf_sagehen-gsf.oc',
    'sagehenmodel/gwf_sagehen-gsf.uzf',
]
for ff in cp_list:
    shutil.copy2(root_dir.parent / f'ModflowPRMS/{ff}', root_dir / ff)
    
all_common = sorted((root_dir.parent / 'ModflowPRMS/common').glob('*'))
for cc in all_common: 
    shutil.copy2(cc, (root_dir / 'common') / cc.name)

In [ ]:
# this requires all the inputs and also the output directory above to exist.
mf6_config_file = 'mfsim.nam'
mf6_dll = '/Users/jamesmcc/usgs/mf6bmipaper/models/bin/libmf6.dylib'
mf6 = ModflowApi(mf6_dll, working_directory=os.getcwd())
mf6.initialize(mf6_config_file)

# MODFLOW 6 time loop
current_time = mf6.get_current_time()
end_time = mf6.get_end_time()
print(f'MF current_time: {current_time}, prms control.start_time: {control.start_time}')
print(f'MF end_time: {end_time}, prms control.n_times: {control.n_times}')

#### Get pointers to MODFLOW 6 variables

In [ ]:
# get pointer to UZF variables
mf6_var_model_dict = {'SINF': 'UZF-1', 'PET': 'UZF-1', 'RUNOFF': 'SFR-1'}
mf6_vars = {}
for vv, mm in mf6_var_model_dict.items():
    mf6_vars[vv] = mf6.get_value_ptr(mf6.get_var_address(vv, name.upper(), mm))

for vv, dd in mf6_vars.items(): 
    print(f'shape of {vv}: {dd.shape}')

#### Run the models

In [ ]:
# control is prms.control
n_time_steps = control.n_times
# n_time_steps = 2 * 365
for istep in range(n_time_steps):
        
    prms.advance()
    
    if control.current_dowy == 0:
        if istep > 0: 
            print('\n')
        print(f"Water year: {control.current_year + 1}")

    stdout_str = f"Day of water year: {str(control.current_dowy + 1).zfill(3)}"   
    print(stdout_str, end='\r')

    # run pynhm
    prms.calculate()
    
    hru_ppt = prms.processes['PRMSAtmosphere'].hru_ppt.current

    potet = prms.processes['PRMSSoilzone'].potet 
    actet = prms.processes['PRMSSoilzone'].hru_actet
    unused_pet = potet - actet

    soil_infil = prms.processes['PRMSSoilzone'].ssres_in + prms.processes['PRMSSoilzone'].pref_flow_infil
    recharge = prms.processes['PRMSSoilzone'].ssr_to_gw + prms.processes['PRMSSoilzone'].soil_to_gw
    
    sroff = prms.processes['PRMSRunoff'].sroff
    interflow = prms.processes['PRMSSoilzone'].ssres_flow
    prms_ro = (sroff + interflow) * in2m * hru_area_m2
    
    # save PRMS results (converted to m3/d) 
    nhm_var_dict['time_out'][istep] = control.current_time
    nhm_var_dict['ppt_out'][istep, :] = hru_ppt * in2m * hru_area_m2
    nhm_var_dict['potet_out'][istep, :] = potet * in2m * hru_area_m2
    nhm_var_dict['actet_out'][istep, :] = actet * in2m * hru_area_m2
    nhm_var_dict['soilinfil_out'][istep, :] = soil_infil * in2m * hru_area_m2
    nhm_var_dict['runoff_out'][istep,:] = sroff * in2m * hru_area_m2
    nhm_var_dict['interflow_out'][istep,:] = interflow * in2m * hru_area_m2
    
    # map runoff to SFR
    mf6_vars['RUNOFF'][:] = hru2mf6(sfrw, prms_ro)  # sroff + ssres_flow
    # map groundwater recharge to MODFLOW
    mf6_vars['SINF'][:nuzf_infilt] = hru2mf6(uzfw, recharge) * in2m  # ssr_to_gw + soil_to_gw
    # map unused pet to MODFLOW
    mf6_vars['PET'][:nuzf_infilt] = hru2mf6(uzfw, unused_pet) * in2m  # potet - actet

    # run MODFLOW 6
    mf6.update()

### Finalize models

In [ ]:
# cleanup
try:
    mf6.finalize()
    prms.finalize()
    success = True
except:
    raise RuntimeError

#### Save PRMS output

In [ ]:
fpth = "output/pynhm_output.npz"
np.savez_compressed(
    fpth,
    time=nhm_var_dict['time_out'], 
    ppt=nhm_var_dict['ppt_out'], 
    potet=nhm_var_dict['potet_out'], 
    actet=nhm_var_dict['actet_out'], 
    infil=nhm_var_dict['soilinfil_out'],
    runoff=nhm_var_dict['runoff_out'],
    interflow=nhm_var_dict['interflow_out'],
)